### Startup imports

In [106]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import slurm_manager

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [107]:
sm=slurm_manager(user='zuranski', host='della.princeton.edu')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [108]:
my_smiles=[
    "",
    "",
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [109]:
my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
my_smiles

['CCC', 'CCCP', 'CCCO']

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [110]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, max_num_conformers=3))

INFO:autoqchem.molecule:Initializing molecule with canonical smiles: CCC
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 1 conformations of CCC molecule
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: CCCP
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 3 conformations of CCCP molecule
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: CCCO
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 3 conformations of CCCO molecule


Draw some molecules just for fun

In [139]:
mols[1].draw(2)

### Create gaussian jobs for each conformation

In [113]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 3 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 3 conformations.


Where is this all happening?? where are the input files?? 

Just like most of your applications it uses the 'AppData' directories, they will differ depending on the OS

In [114]:
sm.workdir

'C:\\Users\\AndrzejZuranski\\AppData\\Local\\autoqchem\\della'

In [138]:
sm.remote_dir

'/home/zuranski/gaussian'

In [115]:
sm.get_job_stats(split_by_can=True)

status,created
can,
CCC,1
CCCO,3
CCCP,3


### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [116]:
sm.submit_jobs()

INFO:autoqchem.slurm_manager:Submitting 7 jobs.
INFO:autoqchem.slurm_manager:Creating connection to della.princeton.edu as zuranski
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)




Password: ········


INFO:paramiko.transport:Authentication (keyboard-interactive) successful!


INFO:autoqchem.slurm_manager:Connected to della.princeton.edu as zuranski.
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:autoqchem.slurm_manager:Submitted job b789590fe3d6ff2a0cdffd45113bd0c7, job_id: 28752734.
INFO:autoqchem.slurm_manager:Submitted job 1122392e48c5e379689d48717e8125ec, job_id: 28752735.
INFO:autoqchem.slurm_manager:Submitted job 9e7d2b1dff7d548bc9275c1902d08c5b, job_id: 28752736.
INFO:autoqchem.slurm_manager:Submitted job d73aa63b8e2379f7bf33ffc7724cb7b1, job_id: 28752737.
INFO:autoqchem.slurm_manager:Submitted job 9a00c301e3fe7c8d856495d371a2726b, job_id: 28752738.
INFO:autoqchem.slurm_manager:Submitted job f26a51025b84f583b965c1b51ff5af01, job_id: 28752739.
INFO:autoqchem.slurm_manager:Submitted job 32c630d8df6a16618e390a156c680585, job_id: 28752740.


Check the status of the jobs, locally and remotely

In [140]:
sm.get_job_stats(split_by_can=True)

status
can


In [133]:
sm.squeue(summary=False)

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [134]:
sm.retrieve_jobs()

INFO:autoqchem.slurm_manager:There are 0 running/pending jobs, 7 finished jobs.
INFO:autoqchem.slurm_manager:Retrieving log files of finished jobs.
INFO:autoqchem.slurm_manager:7 jobs finished successfully (all Gaussian steps finished normally). 0 jobs failed.


### Resubmit failed jobs

In [137]:
sm.resubmit_failed_jobs()

INFO:autoqchem.slurm_manager:There are no failed jobs to resubmit.


### Upload molecules to the database

In [136]:
sm.upload_done_molecules_to_db(tag="tutorial")

INFO:autoqchem.slurm_manager:There are 3 finished molecules ['CCC', 'CCCO', 'CCCP'].
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: CCC, number of conformers: 1.
INFO:autoqchem.slurm_manager:Molecule CCCO has 0 / 3 duplicate conformers.
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: CCCO, number of conformers: 3.
INFO:autoqchem.slurm_manager:Molecule CCCP has 0 / 3 duplicate conformers.
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: CCCP, number of conformers: 3.
